# Image Classification of Dental Xray by @Sakthi
### Steps
1. Import Packages
2. Load Data
3. Build Model
4. Test Model

# Import Packages
#### Importing relevant packages and identifying the necessary folders on computer

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
import pandas as pd

Datadir = r"D:\Jupy\xrays database"
Categories = ["1 root", "2 or more roots"]

# Load Data
#### Resize them to the same shape and size in the event they are not all in the same shape or size
#### Prepare Training dataset

In [2]:
training_data = []
IMG_SIZE = (130, 230)

def create_training_data():
    for category in Categories:
        path = os.path.join(Datadir, category) #path to images
        class_num = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, IMG_SIZE)
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

In [3]:
random.shuffle(training_data) #shuffles image dataset

In [4]:
X= []
y= []

In [5]:
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, 130, 230, 1)
y = np.array(y)
X = X/255.0

## Build Model
#### The following combinations of layers yield the best accuracy to loss ratio as analysed on Tensorboard

In [6]:
dense_layers = [1]
layers_sizes = [128]
convo_layers = [1]

for dense_layer in dense_layers:
    for layers_size in layers_sizes:
        for convo_layer in convo_layers:
            Name = "{}-convo-{}-nodes-{}-dense-{}".format(convo_layer,layers_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir = 'logs/{}'.format(Name))
            model = Sequential()
            model.add(Conv2D(layers_size, (3, 3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size = (2,2)))
            
            for l in range(convo_layer-1):
                
                model.add(Conv2D(layers_size, (3, 3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size = (2,2)))

            model.add(Flatten())
            
            for i in range(dense_layer):
                model.add(Dense(layers_size))
                model.add(Activation("relu"))
                model.add(Dropout(0.2))

            
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss = "binary_crossentropy", 
                         optimizer = "adam", 
                         metrics = ['accuracy'])
            
            model.fit(X,y, batch_size=3,epochs = 10, validation_split=0.1, callbacks = [tensorboard])

Epoch 1/10
 1/35 [..............................] - ETA: 0s - loss: 0.7164 - accuracy: 0.3333WARNING:tensorflow:From C:\Users\Sakthivel\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
35/35 [==============================] - 33s 948ms/step - loss: 3.7286 - accuracy: 0.4563 - val_loss: 0.8024 - val_accuracy: 0.5000
Epoch 2/10
35/35 [==============================] - 36s 1s/step - loss: 0.5526 - accuracy: 0.7282 - val_loss: 0.5284 - val_accuracy: 0.7500
Epoch 3/10
35/35 [==============================] - 32s 914ms/step - loss: 0.2515 - accuracy: 0.9223 - val_loss: 0.3186 - val_accuracy: 0.9167
Epoch 4/10
35/35 [==============================] - 30s 870ms/step - loss: 0.2335 - accuracy: 0.8932 - val_loss: 0.5293 - val_accuracy: 0.6667
Epoch 5/10
35/35 [==============================] - 31s 881ms/s

### Output of model Analysis 
#### Based on the output the model had an epoch accuracy of roughly 0.9806 and epoch loss of 0.0507

In [7]:
model.save('tooth.cnn.model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: tooth.cnn.model\assets


# Test (For future use)
### Create a function to prepare the image for our model and create a method to accept input

In [ ]:
model = tf.keras.models.load_model("tooth.cnn.model")

#For future folder that will be shared
Test_dir = r"D:\Jupy\xrays database\tester"

testing_imgdata = []
testing_catdata = []

def test_data():
    for category in Categories:
        path = os.path.join(Test_dir, category) #path to images
        class_num = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, IMG_SIZE)
                testing_imgdata.append(new_array)
                testing_catdata.append(class_num)
            except Exception as e:
                pass

test_data()
testing_imgdata = np.array(testing_imgdata).reshape(-1, 130, 230, 1)/255.0
testing_catdata = np.array(testing_catdata)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

prediction= model.predict([testing_imgdata])
predicted_val = [int(round(p[0])) for p in prediction]

print("Confusion Matirx:\n ", confusion_matrix(testing_catdata, predicted_val))
print("Classification report:\n ", classification_report(testing_catdata, predicted_val))
